In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import igraph as ig

# Load the graph

In [ ]:
G = ig.read("data/bitcoin/bitcoinotc.graphml")

In [ ]:
G.to_undirected(combine_edges="first")

# Clique distribution

In [ ]:
G.clique_number()

In [ ]:
cl = G.cliques()

In [ ]:
plt.hist(list(map(len,cl)), bins=10);

# Rescale the timestamps

In [ ]:
edges = pd.DataFrame([e.attributes().values() for e in G.es])
edges.columns = G.es.attribute_names()
edges.drop(["Edge Label", "id"], axis=1, inplace=True)
edges.describe()

In [ ]:
timerange = edges.time.max() - edges.time.min()
timemin = edges.time.min()
G.es["time"] = (G.es["time"] - timemin) / timerange

# Temporal subgraphs

In [ ]:
G.subgraph_edges(G.es(lambda e: e["time"] < 0.1)).summary()

In [ ]:
def temporal_subgraph(graph, tmin=None, tmax=None, delete_vertices=True):
    if tmin==None:
        tmin = min(graph.es["time"])
    if tmax==None:
        tmax = max(graph.es["time"])
    return G.subgraph_edges(G.es(lambda e: (e["time"] > tmin) & (e["time"] < tmax)), delete_vertices=delete_vertices)

In [ ]:
temporal_subgraph(G, tmin=0.1, tmax=0.3).summary()

In [ ]:
ti = 0
dt = 0.01
subg = temporal_subgraph(G, tmin=ti, tmax=ti+dt)
layout = subg.layout("kk")
ig.plot(subg, layout=layout)

# Export multilayer graph

In [ ]:
dt = 0.05
layout = []
subg = []
for i in range(2):
    subg.append(temporal_subgraph(G, tmin=i*dt, tmax=i*dt+dt))
    layout.append(subg[i].layout("kk").coords)

In [ ]:
vs_df = pd.DataFrame()
es_df = pd.DataFrame()
for i in range(2):
    subg_vs = pd.DataFrame(layout[i])
    subg_vs.columns = ["x","y"]
    subg_vs["id"] = np.arange(len(layout[i]))
    subg_vs["layer"] = i+1
    # subg_vs.to_csv("vs1.csv", index=False)
    vs_df = pd.concat([vs_df,subg_vs])
    subg_es = pd.DataFrame([[e.source,e.target] for e in subg[i].es])
    subg_es.columns = ["u","v"]
    # subg_es.to_csv("es1.csv", index=False)
    es_df = pd.concat([es_df,subg_es])
    
vs_df.to_csv("vs.csv", index=False)
es_df.to_csv("es.csv", index=False)